In [113]:
import scrapy
import requests
import time
import shapely
import json
from shapely.geometry import asShape
from collections import defaultdict

## Dataset 1 - MTA Geo data

The MTA provides GeoJSON of all the stations, where they are on a map, and what lines they service.

Dataset quirks:
* Multiple stations have the same name (e.g. 103rd st)
* Does not directly contain elevator / escalator data

`feature['properties']['objectid']` is a unique identifier, and
`feature['geometry']` is a valid GeoJSON point.

In [33]:
station_geo=requests.get('https://data.cityofnewyork.us/api/geospatial/arq3-7z49?method=export&format=GeoJSON').json()

In [34]:
mta_stations_sorted_by_name = sorted(station_geo['features'],key=lambda r:r['properties']['name'])
mta_stations_sorted_by_name[0]

{'geometry': {'coordinates': [-73.96837899960818, 40.799446000334825],
  'type': 'Point'},
 'properties': {'line': '1',
  'name': '103rd St',
  'notes': '1-all times',
  'objectid': '159',
  'url': 'http://web.mta.info/nyct/service/'},
 'type': 'Feature'}

## Dataset 2 - Station data from NYCAccessible. 

A non-NYC.gov developer has been regularly scraping elevator outage data 
from http://advisory.mtanyct.info/EEoutage/ to produce JSON of current outages.

He has two useful datasets: 
First, `http://www.nycaccessible.com/api/v1/stations/` - all stations, matched to the number
of escalators and elevators they have and current outages;
Second, `http://www.nycaccessible.com/api/v1/stations/<stn>.json` - for an individual
station, the details of the machines (elevator or escalator) it has, and their outage status.

In [37]:
nyca_stations = requests.get("http://www.nycaccessible.com/api/v1/stations/").json()
nyca_stations[0]

{'accessible_note': None,
 'elevator_count_words': None,
 'escalator_count_words': None,
 'has_machines': False,
 'id': 459,
 'is_accessible': False,
 'lines': ['6'],
 'name': '103rd St',
 'outages': None}

In [44]:
def get_nyca_station_detail(station_id):
    time.sleep(0.2) # Hack to avoid hitting API limits; should do exponential backoff here but too lazy
    detail = requests.get(
        "http://www.nycaccessible.com/api/v1/stations/{}.json".format(station_id)).json()
    return detail
all_nyca_station_details = {stn['id']:get_nyca_station_detail(stn['id']) for stn in nyca_stations}

In [46]:
all_nyca_station_details[1].keys()

dict_keys(['id', 'name', 'lines', 'is_accessible', 'has_machines', 'elevator_count_words', 'escalator_count_words', 'outages', 'accessible_note', 'updated_at', 'machines', 'bus_info'])

In [48]:
all_nyca_station_details[1]['machines'][1]

{'human_eq_type': 'Elevator',
 'image': 'EL_operating.png',
 'lines': ['A', 'B', 'C', 'D', 'E', 'F', 'M'],
 'location': 'NE corner of West 3rd Street and Sixth Avenue',
 'out_since': False,
 'return_date_display': False,
 'serving': 'Street To Mezzanine'}

## Dataset 3 - NYC neighborhood data

Since we'd like to highlight when certain neighborhoods are underserved by accessibility, we need a source
of neighborhood data. Here's one that seeems to be reasonably comprehensive:

http://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/nynta/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=geojson

(Linked from nyc.gov https://www1.nyc.gov/site/planning/data-maps/open-data/dwn-nynta.page )

`feature['properties']['NTACode']` is a unique identifier, and
`feature['geometry']` is a valid GeoJSON polygon.

In [49]:
neighborhood_data=requests.get('http://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/nynta/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=geojson').json()

In [56]:
neighborhood_data['features'][0]['properties']

{'BoroCode': 3,
 'BoroName': 'Brooklyn',
 'CountyFIPS': '047',
 'NTACode': 'BK88',
 'NTAName': 'Borough Park',
 'OBJECTID': 1,
 'Shape__Area': 0.000534040202917259,
 'Shape__Length': 0.12354783848794}

## OK, let's get to work

Starting question: How many subway stops are in each neighborhood?
There's probably a cheaper way to calculate this than `O(n**2)` but I'm under time pressure
so let's do that.

A neighborhood is represented by `neighborhood_data['features'][n]['geometry']`, a Polygon.
A subway stop is represented by `mta_stations_sorted_by_name[n]['geometry']`, a Point.

In [80]:
neighborhood_by_ntacode = {
    n['properties']['NTACode']:n 
    for n in neighborhood_data['features']}
neighborhood_polygons_by_ntacode={
    n['properties']['NTACode']:asShape(n['geometry']) 
    for n in neighborhood_data['features']
}

In [81]:
station_by_objectid = {
    s['properties']['objectid']:s 
    for s in mta_stations_sorted_by_name
}
station_points_by_objectid = {
    s['properties']['objectid']:asShape(s['geometry'])
    for s in mta_stations_sorted_by_name
}


In [ ]:
i=0
total=len(neighborhood_by_ntacode.keys())
def stations_in_neighborhood(ntacode):
    global i
    npoly = neighborhood_polygons_by_ntacode[ntacode]
    nname = neighborhood_by_ntacode[ntacode]['properties']['NTAName']
    print("{i}/{total}: {nname}".format(i=i,total=total,nname=nname))
    i=i+1
    return (ntacode,nname,[
        station_id # objectid
        for station_id,station_point in station_points_by_objectid.items()
        if npoly.contains(station_point)
    ])

neighborhood_to_stations = [
    stations_in_neighborhood(n_id)
    for n_id,n_detail in neighborhood_by_ntacode.items()
]

len(neighborhood_to_stations)
# This takes about 5 minutes to run. Wait....

In [105]:
# invert this to verify - do stations appear in multiple neighborhoods?
stations_to_neighborhoods=defaultdict(lambda:[])
for n_id,n_name,stations in neighborhood_to_stations:
    for s in stations:
        stations_to_neighborhoods[s].append(n_id)
len(stations_to_neighborhoods)

473

In [116]:
# ... No they don't, so the last thing I need is to go back to the original neighborhood
# data and populate it with a count_of_subway_stations property for graphing
neighborhood_to_count = {
    n_id:len(stations)
    for n_id,n_name,stations in neighborhood_to_stations
}
# Enrich the original neighborhood_data geojson with count_of_subway_stations:
for n in neighborhood_data['features']:
    n_id=n['properties']['NTACode']
    n['properties']['count_of_subway_stops'] = neighborhood_to_count[n_id]


In [119]:
neighborhood_data['features'][0]['properties']

{'BoroCode': 3,
 'BoroName': 'Brooklyn',
 'CountyFIPS': '047',
 'NTACode': 'BK88',
 'NTAName': 'Borough Park',
 'OBJECTID': 1,
 'Shape__Area': 0.000534040202917259,
 'Shape__Length': 0.12354783848794,
 'count_of_subway_stops': 4}

## Break time! Let's write our key data structures at this point to json, so others can use them.

In [120]:
with open('neighborhood_to_stations.json','w') as outfile:
    json.dump(neighborhood_to_stations,outfile)
with open('stations_to_neighborhoods.json','w') as outfile:
    json.dump(stations_to_neighborhoods,outfile)
with open('mta_stations_sorted_by_name.json','w') as outfile:
    json.dump(mta_stations_sorted_by_name,outfile)
with open('nyca_stations.json','w') as outfile:
    json.dump(nyca_stations,outfile)
with open('all_nyca_station_details.json','w') as outfile:
    json.dump(all_nyca_station_details,outfile)
with open('neighborhood_data.json','w') as outfile:
    json.dump(neighborhood_data,outfile)